# 1. Import the necessary libraries and read the provided CSVs as a data frame 

In [29]:
import pandas as pd
import os
import glob
from surprise import SVD
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from collections import defaultdict
import seaborn as sns

1.A. Merge all the provided CSVs into one dataFrame.

In [2]:
path = os.getcwd()
path

'/Users/gobinathvelusamy'

In [3]:
csv_files = glob.glob(os.path.join('data/', "*.csv"))

In [4]:
csv_files

['data/phone_user_review_file_1.csv',
 'data/phone_user_review_file_2.csv',
 'data/phone_user_review_file_3.csv',
 'data/phone_user_review_file_6.csv',
 'data/phone_user_review_file_4.csv',
 'data/phone_user_review_file_5.csv']

In [5]:
for f in csv_files:
    df = pd.read_csv(f,encoding='latin-1')

B2.. Explore, understand the Data and share at least 2 observations. 

In [6]:
df.shape

(350216, 11)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350216 entries, 0 to 350215
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   phone_url  350216 non-null  object 
 1   date       350216 non-null  object 
 2   lang       350216 non-null  object 
 3   country    350216 non-null  object 
 4   source     350216 non-null  object 
 5   domain     350216 non-null  object 
 6   score      321983 non-null  float64
 7   score_max  321983 non-null  float64
 8   extract    341836 non-null  object 
 9   author     321351 non-null  object 
 10  product    350216 non-null  object 
dtypes: float64(2), object(9)
memory usage: 29.4+ MB


In [8]:
df.describe()

,score,score_max
count,321983.000000,321983.0
mean,7.773498,10.0
std,2.644970,0.0
min,0.200000,10.0
25%,6.000000,10.0
50%,8.000000,10.0
75%,10.000000,10.0
max,10.000000,10.0


In [9]:
df.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,2.0,10.0,I bought 1 month before. currently speaker is ...,venkatesh,Karbonn K1616
1,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,6.0,10.0,"I just bought one week back, I have Airtel con...",Venkat,Karbonn K1616
2,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,4.0,10.0,one problem in this handset opera is not worki...,krrish,Karbonn K1616
3,/cellphones/karbonn-k1616/,4/25/2014,en,in,Naaptol,naaptol.com,10.0,10.0,here Karbonn comes up with an another excellen...,BRIJESH CHAUHAN,Karbonn K1616 - Black
4,/cellphones/karbonn-k1616/,4/23/2013,en,in,Naaptol,naaptol.com,10.0,10.0,"What a phone, all so on Naaptol my god 23% off...",Suraj CHAUHAN,Karbonn K1616 - Black


Observation : 
1.Data set has 350216 rows and 11 Columns 
2.Score & Score_max columns has integer data type remaning Columns are categorical 
3.Score_max has uunique value 10 . So std deviationas Zero
4.Mean values of Score will be 7.773498

1.C. Round off scores to the nearest integers. 

In [11]:
df[['score','score_max']] = df[['score','score_max']].round().astype('Int64')

In [12]:
pd.set_option("display.max_rows", None)

In [13]:
df.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,2,10,I bought 1 month before. currently speaker is ...,venkatesh,Karbonn K1616
1,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,6,10,"I just bought one week back, I have Airtel con...",Venkat,Karbonn K1616
2,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,4,10,one problem in this handset opera is not worki...,krrish,Karbonn K1616
3,/cellphones/karbonn-k1616/,4/25/2014,en,in,Naaptol,naaptol.com,10,10,here Karbonn comes up with an another excellen...,BRIJESH CHAUHAN,Karbonn K1616 - Black
4,/cellphones/karbonn-k1616/,4/23/2013,en,in,Naaptol,naaptol.com,10,10,"What a phone, all so on Naaptol my god 23% off...",Suraj CHAUHAN,Karbonn K1616 - Black


1.D. Check for missing values. Impute the missing values, if any. 

In [14]:
df.isnull().sum() / df.shape[0] * 100.00

phone_url    0.000000
date         0.000000
lang         0.000000
country      0.000000
source       0.000000
domain       0.000000
score        8.061596
score_max    8.061596
extract      2.392809
author       8.242056
product      0.000000
dtype: float64

In [15]:
# filling the null values in column 'score' and 'score_max' as these are numerical Values
df = df.fillna(df.median())

/var/folders/7p/5k1s0qxj2mn1kmk2d_npxq8r0000gn/T/ipykernel_1117/885693151.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.median())


In [16]:
df.isnull().sum() / df.shape[0] * 100.00

phone_url    0.000000
date         0.000000
lang         0.000000
country      0.000000
source       0.000000
domain       0.000000
score        0.000000
score_max    0.000000
extract      2.392809
author       8.242056
product      0.000000
dtype: float64

In [17]:
# dropping the null values in columns 'extract' ,'author' and 'product'
df = df.dropna()

In [18]:
df.isnull().sum() / df.shape[0] * 100.00

phone_url    0.0
date         0.0
lang         0.0
country      0.0
source       0.0
domain       0.0
score        0.0
score_max    0.0
extract      0.0
author       0.0
product      0.0
dtype: float64

1.E.Check for duplicate values and remove them, i

In [19]:
duplicate = df[df.duplicated()]
duplicate

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
866,/cellphones/alcatel-8000d/,7/26/2014,ru,ru,Yandex,market.yandex.ru,4,10,"ÐÐ°ÑÐ°ÑÐµÑ Ñ Ð¾ÑÑ Ð¸ 2500Ð¼ÐÑ, Ð½Ð¾ Ð´...",Ð ÐµÐ¿Ð¸Ð½ ÐÐ¼Ð¸ÑÑÐ¸Ð¹,Alcatel One Touch SCRIBE EASY 8000D
885,/cellphones/alcatel-8000d/,2/11/2014,ru,ru,Yandex,market.yandex.ru,10,10,"Ð¢ÐµÐ»ÐµÑÐ¾Ð½ Ñ Ð¾ÑÐ¾Ñ,Ð¾ÑÐ¾Ð±ÐµÐ½Ð½Ð¾ Ð·Ð...",Ð§ÐµÑÐ½ÑÑÐ¾Ð² ÐÐ»ÐµÐºÑÐ°Ð½Ð´Ñ,Alcatel One Touch SCRIBE EASY 8000D
1411,/cellphones/samsung-galaxy-pocket/,12/30/2012,it,it,Amazon,amazon.it,10,10,"Un bellissimo telefonino, piccolo (ma non trop...",johanna,Samsung Galaxy Pocket S 5300 Nero
2156,/cellphones/lg-a290/,6/26/2013,pt,br,Forum Celular,forumcelular.com.br,10,10,Um dos melhores aparelhos que jÃ¡ comprei. Sim...,jorge ricardo,LG A290
2382,/cellphones/prestigio-multiphone-4000/,3/13/2014,ru,ru,Yandex,market.yandex.ru,6,10,Ð¡Ð¿ÑÑÑÑ Ð¿Ð¾Ð»Ð³Ð¾Ð´Ð° Ð¸ÑÐ¿Ð¾Ð»ÑÐ·Ð¾Ð²...,ÐÐ¸ÐºÐ¾Ð»Ð°ÐµÐ² ÐÐ»ÐµÐºÑÐµÐ¹,Prestigio MultiPhone 4000 DUO
2391,/cellphones/prestigio-multiphone-4000/,7/18/2013,ru,ru,Yandex,market.yandex.ru,8,10,"Ð Ð¿ÑÐ¸Ð½ÑÐ¸Ð¿Ðµ, Ð½ÐµÐ´Ð¾ÑÑÐ°ÑÐ¾Ðº ÑÑ...",thing Killen,Prestigio MultiPhone 4000 DUO
2396,/cellphones/prestigio-multiphone-4000/,6/13/2013,ru,ru,Yandex,market.yandex.ru,8,10,ÐÑÐ¿Ð¸Ð» Ð´Ð°Ð½Ð½ÑÐ¹ Ð°Ð¿Ð¿Ð°ÑÐ°Ñ Ð¿Ð¾ÑÐ...,Ð¦Ð°ÑÑÐ² Ð®ÑÐ°,Prestigio MultiPhone 4000 DUO
3384,/cellphones/nokia-asha-203/,4/15/2013,ru,ru,Yandex,market.yandex.ru,8,10,ÐÐ¾Ð½ÑÐ°Ð²Ð¸Ð»ÑÑ Ð² Ð¾Ð±ÑÐµÐ¼ Ð¸ ÑÐµÐ»Ð¾...,sivirina2006,Nokia Asha 203
3451,/cellphones/nokia-asha-203/,12/19/2012,it,it,Amazon,amazon.it,10,10,Il Nokia Asha 203 Ã¨ un cellulare dalle poche ...,Edoardo,"Nokia 203, Bianco [Italia]"
3455,/cellphones/nokia-asha-203/,11/14/2012,it,it,Amazon,amazon.it,10,10,"Ottimo prodotto di Nokia! Uno smartphone ""econ...",asterix,"Nokia 203, Bianco [Italia]"


We have 2160 Duplicated Records 

In [20]:
df=df.drop_duplicates(keep=False)

In [21]:
duplicate = df[df.duplicated()]
duplicate

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product


1.F. Keep only 1 Million data samples. Use random state=612.

In [22]:
df1 = df.sample(n=1000000, random_state=612,replace=True)

In [23]:
df1.shape

(1000000, 11)

In [24]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 186305 to 346274
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   phone_url  1000000 non-null  object
 1   date       1000000 non-null  object
 2   lang       1000000 non-null  object
 3   country    1000000 non-null  object
 4   source     1000000 non-null  object
 5   domain     1000000 non-null  object
 6   score      1000000 non-null  Int64 
 7   score_max  1000000 non-null  Int64 
 8   extract    1000000 non-null  object
 9   author     1000000 non-null  object
 10  product    1000000 non-null  object
dtypes: Int64(2), object(9)
memory usage: 93.5+ MB


 G. Drop irrelevant features. Keep features like Author, Product, and Score.

In [25]:
df2=df1.drop(['phone_url','date','lang','country','source','domain','score_max','extract'],axis=1)

In [26]:
df2.head(10)

,score,author,product
186305,8,chuqueno,NOKIA 7230
297238,8,dunya,Blackberry Storm 9530 Cep Telefonu
234982,10,Amazon Kunde,Microsoft Nokia 5800 XpressMusic Comes with Mu...
207714,2,Evren Yurtesen,Motorola Milestone 2 Unlocked Cell Phone with ...
274902,9,kimmy4vols,VX-8360
314871,2,Luis ogando C,Samsung Galaxy S II i777 16GB GSM Android Smar...
303659,6,ANDERSONETALITA2010,BlackBerry Bold 9000
14098,8,Wand,LG P690 Optimus Net
309754,10,rammstein95,Nokia 3120 Classic
314737,10,SERGIOV89,LG KP265


# 2 Part 

2.A. Identify the most rated products. 

In [34]:
df2.groupby('product')['score'].mean().sort_values(ascending=False).head()  

product
Nokia E63 Bleu outremer Symbian OS 9.2 / Series 60 3.1 Edition                                                                     10.0
BlackBerry Curve 9300 Zwart                                                                                                        10.0
Sony Ericsson LT15IEUBLU XPERIA Arc Android Smartphone with Touchscreen, 8MP Camera, Bluetooth, Wi-Fi, GPS - No Warranty - Blue    10.0
HTC HD2 Zwart                                                                                                                      10.0
The Sony Ericsson Xperia X10                                                                                                       10.0
Name: score, dtype: Float64

2.B Identify the users with most number of reviews.

In [63]:
(df2['author'].value_counts()).head()

Amazon Customer    12977
Anonymous           3121
e-bit               2289
Anonyme             1241
ÐÐ¾ÑÑÑ          1212
Name: author, dtype: int64

C. Select the data with products having more than 50 ratings and users who have given more than 50 ratings.

In [65]:
df2['product'].value_counts().head()

Nokia 5800 XpressMusic    5181
Nokia 5800                5002
Nokia N8                  4602
LG GX200                  4565
Nokia E71                 3918
Name: product, dtype: int64

In [66]:
#extracting authors who gave greater than 50 ratings
df_a = pd.DataFrame(columns=['author', 'a_count'])

In [67]:
df_a

,author,a_count


In [68]:
df2.shape

(1000000, 3)

In [69]:
df_a['author']=df2['author'].value_counts().index.tolist() 
df_a['a_count'] = list(df2['author'].value_counts() > 50)

In [70]:
# get names of indexes for which count column value is False
index_names = df_a[ df_a['a_count'] == False ].index 
# drop these row indexes from dataFrame 
df_a.drop(index_names, inplace = True) 
df_a

,author,a_count
0,Amazon Customer,True
1,Anonymous,True
2,e-bit,True
3,Anonyme,True
4,ÐÐ¾ÑÑÑ,True
5,Misafir,True
6,Amazon Kunde,True
7,zehrabalta,True
8,kuzeykizi,True
9,Alex,True


In [71]:
df_p = pd.DataFrame(columns=['product', 'p_count'])
df_p['product']=df2['product'].value_counts().index.tolist() 
df_p['p_count'] = list(df2['product'].value_counts() > 50)

In [72]:
# get names of indexes for which count column value is False
index_names = df_p[ df_p['p_count'] == False ].index 
# drop these row indexes from dataFrame 
df_p.drop(index_names, inplace = True)


In [ ]:
df_p

In [73]:
df_p1 = df2[df2['product'].isin(df_p['product'])] 

In [74]:
df_a1 = df_p1[df_p1['author'].isin(df_a['author'])]

In [ ]:
df_a1

In [75]:
df_a1.shape

(87273, 3)

# 3. Build a popularity based model and recommend top 5 mobile phones

In [76]:
#calculating the mean score for a product by grouping it.
ratings_mean_count = pd.DataFrame(df2.groupby('product')['score'].mean()) 

In [77]:
# calculating the number of ratings a product got
ratings_mean_count['rating_counts'] = pd.DataFrame(df2.groupby('product')['score'].count())  

In [78]:
# 3. Recommending the 5 mobile phones based in highest mean score and highest number of ratings the product got. 
ratings_mean_count.sort_values(by=['score','rating_counts'], ascending=[False,False]).head()

,score,rating_counts
product,,
Nokia : SÃÂ­ : SÃÂ­ 6630,10.0,55
"Doro Phone easy 615 Big Button SIM Free Mobile Phone, Black",10.0,44
Samsung i9195 Galaxy S4 mini Mobiltelefon,10.0,40
Ð¢ÐµÐ»ÐµÑÐ¾Ð½ LG E400 Optimus L3 Black,10.0,39
Ð¢ÐµÐ»ÐµÑÐ¾Ð½ LG P970 Optimus Black Titanium Black,10.0,38


# 4. Build a collaborative filtering model using SVD.

In [79]:
# arranging columns in the order of user id,item id and rating to be fed in the svd
columns_titles = ['author','product','score']
vs_rev = df2.reindex(columns=columns_titles)

In [80]:
# Keep only 5000 data samples. Use random state=612
vs_data = vs_rev.sample(n=5000, random_state=612)

In [81]:
# 4. Build a collaborative filtering model using SVD. 
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(vs_data,reader = reader)

In [82]:
trainset = data.build_full_trainset()

In [83]:
trainset.ur


defaultdict(list,
            {0: [(0, 8.0)],
             1: [(1, 8.0)],
             2: [(2, 10.0)],
             3: [(3, 8.0)],
             4: [(4, 8.0)],
             5: [(5, 10.0)],
             6: [(6, 10.0)],
             7: [(7, 10.0)],
             8: [(8, 10.0)],
             9: [(9, 10.0)],
             10: [(10, 10.0)],
             11: [(11, 8.0)],
             12: [(12, 8.0)],
             13: [(13, 8.0)],
             14: [(14, 2.0)],
             15: [(15, 10.0)],
             16: [(16, 6.0)],
             17: [(17, 6.0)],
             18: [(18, 8.0)],
             19: [(19, 10.0)],
             20: [(20, 8.0)],
             21: [(21, 10.0)],
             22: [(22, 8.0)],
             23: [(23, 8.0)],
             24: [(24, 10.0)],
             25: [(25, 10.0)],
             26: [(26, 10.0)],
             27: [(27, 2.0)],
             28: [(28, 8.0)],
             29: [(29, 6.0), (2241, 4.0)],
             30: [(30, 8.0)],
             31: [(31, 10.0), (218, 10.0), (21

In [84]:
algo = SVD()
algo.fit(trainset)

In [85]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()

In [86]:
predictions = algo.test(testset)

In [87]:
predictions

[Prediction(uid='Frota', iid='SONY ERICSSON Vivaz', r_ui=7.7382, est=7.6496287978302115, details={'was_impossible': False}),
 Prediction(uid='Frota', iid='HP Slate 6 Voice Tab (6inch,16GB,Wi-Fi+3G+Voice Calling), Graphite', r_ui=7.7382, est=7.359374529801991, details={'was_impossible': False}),
 Prediction(uid='Frota', iid='Samsung Wave 3 GT-S8600', r_ui=7.7382, est=7.829761401088123, details={'was_impossible': False}),
 Prediction(uid='Frota', iid='Apple iPhone 3GS 8Go Noir', r_ui=7.7382, est=7.658618720713803, details={'was_impossible': False}),
 Prediction(uid='Frota', iid='Samsung Galaxy S II i9100 DualCore Smartphone (10.9 cm (4.3 Zoll) Super-Amoled Plus Display, Android 4.0 oder hÃ¶her, 8 MP Full-HD Kamera, 2 MP Frontkamera) noble-black', r_ui=7.7382, est=7.8156548463118725, details={'was_impossible': False}),
 Prediction(uid='Frota', iid='Samsung Galaxy S II i9100 16 GB Unlocked GSM Smartphone, Noble Black', r_ui=7.7382, est=7.80024795475928, details={'was_impossible': False}),


Build a collaborative filtering model using kNNWithMeans from surprise using User based model


In [89]:
reader = Reader(rating_scale=(1, 10))
data_U = Dataset.load_from_df(vs_data,reader = reader)

In [90]:
trainset_U, testset_U = train_test_split(data_U, test_size=.15)


In [91]:
# Use user_based true/false to switch between user-based or item-based collaborative filtering
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset_U)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [92]:
# we can now query for specific predicions
uid = 'Frances DeSimone'  # raw user id
iid = 'Samsung Galaxy Star Pro DUOS S7262 Unlocked Ce.'  # raw item id

In [93]:
# get a prediction for specific users and items.
pred = algo.predict(uid, iid, verbose=True)

user: Frances DeSimone item: Samsung Galaxy Star Pro DUOS S7262 Unlocked Ce. r_ui = None   est = 7.74   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


5. Evaluate the collaborative model. Print RMSE value.

In [94]:
print("SVD Model : Test Set")
accuracy.rmse(predictions, verbose=True)

SVD Model : Test Set
RMSE: 0.3719


0.37190047749442706

6. Predict score (average rating) for test users. 

In [95]:
# run the trained model against the testset
test_pred_U = algo.test(testset_U)

In [96]:
test_pred_U

[Prediction(uid='FRANCIS_BRAGA2008', iid='Motorola V9', r_ui=10.0, est=7.736941176470588, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='samir1917', iid='Huawei IDEOS X3', r_ui=10.0, est=7.736941176470588, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='RobTouchPro2', iid='HTC Touch Pro2 CDMA', r_ui=9.0, est=7.736941176470588, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='dennyboomboom', iid='Sony Phones Xperia Go Sim Free Mobile Phone - Black', r_ui=2.0, est=7.736941176470588, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='emanetcan', iid='Nokia 3720 Classic cep telefonu', r_ui=8.0, est=8.0, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid='Ð\x9cÐ°ÐºÑ\x81Ð¸Ð¼ÐµÐ½ÐºÐ¾ ?Ñ\x80Ð¸Ð½Ð° Ð\x9fÐµÑ\x82Ñ\x80?Ð²Ð½Ð°', iid='Nokia 5230 XpressMusic', r_ui=10.0, est=7.736941176470588, details={'was

7. Report your findings and inferences. 

Overall data is highly skewed towards 'Amazon customers' 
Most of the authors have given the rating of '10' or '8'

8. Try and recommend top 5 products for test users. 

In [97]:
def get_top_n(predictions, n=5):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [98]:
# 8. Try and recommend top 5 products for test users
top_n = get_top_n(predictions, n=5)

In [99]:
top_n

defaultdict(list,
            {'Frota': [('HTC Desire S', 8.900019180452748),
              ('Nokia 5800', 8.760514383891639),
              ('LG Optimus L7 P705 Black', 8.72791664151785),
              ('Nokia E63', 8.7237379653512),
              ('Apple iPhone 3GS 32Gb', 8.70911758984763)],
             'papounet59': [('HTC Desire S', 8.910146476850782),
              ('LG Optimus One P500', 8.76772604504313),
              ('Samsung GT-S5230 Star', 8.762989153287107),
              ('Samsung Galaxy S II White', 8.755018452974916),
              ('LG Optimus L7 P705 Black', 8.705643414245184)],
             'JAGAJIVAN': [('Samsung Galaxy S II White', 9.191390703757865),
              ('Samsung Jet S8000', 9.039213527975026),
              ('HTC Desire S', 9.036751881319423),
              ('Nokia 5800', 8.892026355121496),
              ('Nokia E63', 8.877524996602144)],
             'uncommon13': [('Apple iPhone 4 16Gb', 8.972416102084212),
              ('HTC Desire S', 8.86196709

9. Try other techniques (Example: cross validation) to get better results. 

In [100]:
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([2.54097559, 2.50849413, 2.6213206 ]),
 'fit_time': (0.3203132152557373, 0.21360397338867188, 0.17496514320373535),
 'test_time': (0.006541013717651367,
  0.005762815475463867,
  0.005630970001220703)}

Observation :
    RMSE of SVD model is lower than for cross validation.

10. In what business scenario you should use popularity based Recommendation Systems ?

It does not suffer from cold start problems which means on day 1 of the business also it can recommend products on various different filters.

There is no need for the user's historical data.

Popularity based recommendation system relies on the popularity,trends and frequency counts of which items were most purchased.

It is used buy the travel companies selling holiday packages in a season, by Google News and other news websites to show Top Stories with images.

11. In what business scenario you should use CF based Recommendation Systems ?

.Collaborative Filtering is used to building intelligent recommender systems that can learn to give better recommendations as more information about users is collected. It is a  personalised recommender system , recommendations are made based on the past behaviour of the user. Most websites like Amazon, YouTube, and Netflix use collaborative filtering as a part of their sophisticated recommendation system.

12. What other possible methods can you think of which can further improve the recommendation for different users ?

Apart from Popularity and Collaborative Filtering , Content-based, Demographic, Utility based, Knowledge based and Hybrid recommendation system can be used as per the user needs.